### Import packages

In [1]:
import pandas as pd
from datasets import Dataset

### Import dataset from CSV file

In [2]:
data = pd.read_csv('../data/paris-2024-faq.csv',delimiter=";")

### Extract french language from FAQ

In [3]:
# extract french FAQ
data = data[data['lang']=='fr']

### Combine a new text field for semantic search

In [4]:
data['text'] = data['label'] + ' ' + data['body'] + ' ' + data['topics']

### Import sentence transformers from hugging face

In [19]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

/home/farid/.pyenv/versions/3.10.6/envs/dev-go/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Embeddings functions

In [6]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors="pt")
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

### Get embeddings on the text column

In [10]:
text_dataset = Dataset.from_pandas(data)

embeddings_dataset = text_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().numpy()[0]}
)

Map:   0%|          | 0/543 [00:00<?, ? examples/s]

### add a faiss index on embeddings dataset

In [34]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'lang', 'label', 'body', 'topics', 'url', 'text', '__index_level_0__', 'embeddings'],
    num_rows: 543
})

### Question embedding

In [28]:
question = "flamme"
question_embedding = get_embeddings([question]).detach().numpy()
question_embedding.shape

(1, 768)

### Get nearest examples between questions and text 

In [29]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [ ]:
### Display results, look ok

In [36]:
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)
samples_df

,id,lang,label,body,topics,url,text,__index_level_0__,embeddings,scores
4,OngTYmjF-fr,fr,Faut-il être de nationalité française pour por...,"Le Relais de la Flamme, à l’image des Jeux est...",Relais de la Flamme ;Les porteurs de la Flamme,https://help.paris2024.org/contents/Faut-il-et...,Faut-il être de nationalité française pour por...,168,"[-0.07542101293802261, -0.2647593319416046, -0...",37.694481
3,OngUUij5-fr,fr,Faut-il une condition physique particulière po...,Le Relais de la Flamme n’est pas une compétiti...,Relais de la Flamme ;Les porteurs de la Flamme,https://help.paris2024.org/contents/Faut-il-un...,Faut-il une condition physique particulière po...,166,"[-0.17378737032413483, -0.5798210501670837, -0...",37.668285
2,OngiDqKi-fr,fr,D’où vient la Flamme Olympique ? Que représent...,"Dans l’Antiquité, les Grecs considéraient le f...",Relais de la Flamme ;Les Flammes et la torche,https://help.paris2024.org/contents/D-ou-vient...,D’où vient la Flamme Olympique ? Que représent...,169,"[-0.058181218802928925, -0.14278440177440643, ...",36.726715
1,OngjaAN0-fr,fr,Que se passe-t-il si la Flamme Olympique s’éte...,La Flamme Olympique est un élément sacré.\n\nD...,Relais de la Flamme ;Les Flammes et la torche,https://help.paris2024.org/contents/Que-se-pas...,Que se passe-t-il si la Flamme Olympique s’éte...,170,"[-0.06371043622493744, -0.4322930872440338, -0...",36.184418
0,OngdYmoQ-fr,fr,Que devient la Flamme la nuit ?,"De nuit, comme de jour, la Flamme est gardienn...",Relais de la Flamme ;Les Flammes et la torche,https://help.paris2024.org/contents/Que-devien...,"Que devient la Flamme la nuit ? De nuit, comme...",781,"[0.17467954754829407, -0.8337361812591553, -0....",35.397820


,id,lang,label,body,topics,url,text,__index_level_0__,embeddings,scores
4,OngTYmjF-fr,fr,Faut-il être de nationalité française pour por...,"Le Relais de la Flamme, à l’image des Jeux est...",Relais de la Flamme ;Les porteurs de la Flamme,https://help.paris2024.org/contents/Faut-il-et...,Faut-il être de nationalité française pour por...,168,"[-0.07542101293802261, -0.2647593319416046, -0...",37.694481
3,OngUUij5-fr,fr,Faut-il une condition physique particulière po...,Le Relais de la Flamme n’est pas une compétiti...,Relais de la Flamme ;Les porteurs de la Flamme,https://help.paris2024.org/contents/Faut-il-un...,Faut-il une condition physique particulière po...,166,"[-0.17378737032413483, -0.5798210501670837, -0...",37.668285
2,OngiDqKi-fr,fr,D’où vient la Flamme Olympique ? Que représent...,"Dans l’Antiquité, les Grecs considéraient le f...",Relais de la Flamme ;Les Flammes et la torche,https://help.paris2024.org/contents/D-ou-vient...,D’où vient la Flamme Olympique ? Que représent...,169,"[-0.058181218802928925, -0.14278440177440643, ...",36.726715
1,OngjaAN0-fr,fr,Que se passe-t-il si la Flamme Olympique s’éte...,La Flamme Olympique est un élément sacré.\n\nD...,Relais de la Flamme ;Les Flammes et la torche,https://help.paris2024.org/contents/Que-se-pas...,Que se passe-t-il si la Flamme Olympique s’éte...,170,"[-0.06371043622493744, -0.4322930872440338, -0...",36.184418
0,OngdYmoQ-fr,fr,Que devient la Flamme la nuit ?,"De nuit, comme de jour, la Flamme est gardienn...",Relais de la Flamme ;Les Flammes et la torche,https://help.paris2024.org/contents/Que-devien...,"Que devient la Flamme la nuit ? De nuit, comme...",781,"[0.17467954754829407, -0.8337361812591553, -0....",35.397820
